In [1]:
# Import statements and global helper methods
import numpy as np
import nltk
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC, LinearSVC
from tqdm import tqdm_notebook
from gensim.models import word2vec
# from keras.models import Sequential
# from keras.layers import Dense, Activation
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.utils import np_utils
import re

In [2]:
stanford_run = True

In [47]:
if stanford_run:
    # Load in the data sets

    raw_sentences = []

    with open("datasets/datasetSentences.txt", encoding="utf-8") as sentence_file:
        # Skip header line
        next(sentence_file)
        for line in tqdm_notebook(sentence_file):
            sentence = line.split("\t")[1].strip()
            raw_sentences.append(sentence)

    sentiment_map = {}
    with open("datasets/sentiment_labels.txt") as sentiment_file:
        next(sentiment_file)
        for line in tqdm_notebook(sentiment_file):
            index, score = tuple(line.split("|"))
            score = float(score.strip())
            sentiment_map[index] = score

    sentence_to_phrase_map = {}
    with open("datasets/dictionary.txt", encoding="utf-8") as dictionary_file:
        for line in tqdm_notebook(dictionary_file):
            phrase, phrase_num = tuple(line.split("|"))
            sentence_to_phrase_map[phrase] = phrase_num.strip()

    labelled_sentences = [(sentence, sentiment_map[sentence_to_phrase_map[sentence]]) for sentence in raw_sentences]

    splits = ([], [], [])

    with open("datasets/datasetSplit.txt") as split_file:
        next(split_file)
        for line in tqdm_notebook(split_file):
            index, split = tuple(line.split(","))
            index, split = int(index), int(split)
            # Splits are labelled 1-3
            splits[split - 1].append(labelled_sentences[index - 1])

    train_set, test_set, dev_set = splits
    train_sentences, train_y = zip(*train_set)
    dev_sentences, dev_y = zip(*dev_set)
    test_sentences, test_y = zip(*test_set)
    
     #Copy data, for filtering neutral coarse labels
    train_sentences_coarse = train_sentences + tuple()
    dev_sentences_coarse = dev_sentences + tuple()
    test_sentences_coarse = test_sentences + tuple()

In [48]:
if not stanford_run:
    #Method used to load the data from the given files
    def openFile(fileName):
        #open file
        with open(fileName, 'r', encoding = 'utf-8') as file:

            #read the entire file
            data = file.read()

            #split the data into the separate reviews
            data = data.split('\t')

            reviews = []
            scores = []

            #append the first review to list
            reviews.append(data[0])

            #append each review to the list and each score to the previous review
            for review in data[1:-1]:
                reviews.append(review[1:])
                scores.append(review[0])

            #remove trailing \n and add last score
            reviews[-1] = reviews[-1].replace('\n', '')
            scores.append(data[-1].replace('\n', ''))

            return reviews, scores

    #loading all the data
    train_sentences, train_y = openFile('datasets/IMDB-train.txt')
    dev_sentences, dev_y = openFile('datasets/IMDB-valid.txt')
    test_sentences, test_y = openFile('datasets/IMDB-test.txt')

In [49]:
# Tools for feature extraction

# When lemmatizing, we need to convert from NLTK's part of speec
# to wordnet's recognized parts of speech
def get_wordnet_pos(treebank_pos):
    if treebank_pos.startswith('J'):
        return wordnet.ADJ
    elif treebank_pos.startswith('V'):
        return wordnet.VERB
    elif treebank_pos.startswith('N'):
        return wordnet.NOUN
    elif treebank_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def sentence_tokenize(sentence, lem = WordNetLemmatizer()):
    tokens = nltk.word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)
    return [lem.lemmatize(w, pos=get_wordnet_pos(pos)) for (w, pos) in tagged_tokens]

count_vectorizer = CountVectorizer(
    input = "content",
    tokenizer = sentence_tokenize
)

tuple_count_vectorizer = CountVectorizer(
    input = "content",
    tokenizer = sentence_tokenize,
    ngram_range = (1, 2)
)

# Fit all the sentences in the training set
count_vectorizer.fit(train_sentences)
tuple_count_vectorizer.fit(train_sentences)

def count_vectorize(sentences, ngram=False):
    if ngram:
        return tuple_count_vectorizer.transform(sentences)
    else:
        return count_vectorizer.transform(sentences)

In [50]:
# Label conversion
def coarse_label(sentiment):
    if sentiment > 0.6:
        return "Positive"
    elif sentiment < 0.4:
        return "Negative"
    else:
        return "Neutral"
    
def fine_label(sentiment):
    if sentiment <= 0.2:
        return "Very Negative"
    elif sentiment <= 0.4:
        return "Negative"
    elif sentiment <= 0.6:
        return "Neutral"
    elif sentiment <= 0.8:
        return "Positive"
    else:
        return "Very Positive"
    
coarse_train_y = [coarse_label(y) for y in tqdm_notebook(train_y)]
coarse_dev_y = [coarse_label(y) for y in tqdm_notebook(dev_y)]
coarse_test_y = [coarse_label(y) for y in tqdm_notebook(test_y)]

fine_train_y = [fine_label(y) for y in tqdm_notebook(train_y)]
fine_dev_y = [fine_label(y) for y in tqdm_notebook(dev_y)]
fine_test_y = [fine_label(y) for y in tqdm_notebook(test_y)]

# Remove neutral coarse labels and sentences from the earlier copies of sentences
def remove_neutral_labels(coarse_sentences, coarse_labels):
    non_neutral_sentences = ()
    non_neutral_labels = []
    for i in range(len(coarse_labels)):
        if coarse_labels[i] != "Neutral":
            non_neutral_sentences += (coarse_sentences[i],)
            non_neutral_labels.append(coarse_labels[i])
            
            
    return non_neutral_sentences, non_neutral_labels

train_sentences_coarse, coarse_train_y = remove_neutral_labels(train_sentences_coarse, coarse_train_y)
dev_sentences_coarse, coarse_dev_y = remove_neutral_labels(dev_sentences_coarse, coarse_dev_y)
test_sentences_coarse, coarse_test_y = remove_neutral_labels(test_sentences_coarse, coarse_test_y)


The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth .
Singer\/composer Bryan Adams contributes a slew of songs -- a few potential hits , a few more simply intrusive to the story -- but the whole package certainly captures the intended , er , spirit of the piece .
Yet the act is still charming here .
Whether or not you 're enlightened by any of Derrida 's lectures on `` the other '' and `` the self , '' Derrida is an undeniably fascinating and playful fellow .
Just the labour involved in creating the layered richness of the imagery in this chiaroscuro of madness and light is astonishing .
Part of the charm of Satin Rouge is that it avoi

Ms. Seigner and Mr. Serrault bring fresh , unforced naturalism to their characters .
Allen shows he can outgag any of those young whippersnappers making moving pictures today .
A good film with a solid pedigree both in front of and , more specifically , behind the camera .
By no means a slam-dunk and sure to ultimately disappoint the action fans who will be moved to the edge of their seats by the dynamic first act , it still comes off as a touching , transcendent love story .
I encourage young and old alike to go see this unique and entertaining twist on the classic whale 's tale -- you wo n't be sorry !
A literary detective story is still a detective story and aficionados of the whodunit wo n't be disappointed .
High Crimes steals so freely from other movies and combines enough disparate types of films that it ca n't help but engage an audience .
If you 're a fan of the series you 'll love it and probably want to see it twice .
I will be .
It celebrates the group 's playful spark of n

The Pianist is the film Roman Polanski may have been born to make .
This version does justice both to Stevenson and to the sci-fi genre .
Poignant and delicately complex .
Enough may pander to our basest desires for payback , but unlike many revenge fantasies , it ultimately delivers .
Cho 's latest comic set is n't as sharp or as fresh as I 'm the One That I Want ... but it 's still damn funny stuff .
In The Pianist , Polanski is saying what he has long wanted to say , confronting the roots of his own preoccupations and obsessions , and he allows nothing to get in the way .
Despite the film 's shortcomings , the stories are quietly moving .
If you come from a family that eats , meddles , argues , laughs , kibbitzes and fights together , then go see this delightful comedy .
This bracingly truthful antidote to Hollywood teenage movies that slather Clearasil over the blemishes of youth captures the combustible mixture of a chafing inner loneliness and desperate grandiosity that tend to c

Schütte 's dramatic snapshot of the artist three days before his death offers an interesting bit of speculation as to the issues Brecht faced as his life drew to a close .
A slick , engrossing melodrama .
S1M0NE 's satire is not subtle , but it is effective .
It 's a quirky , off-beat project .
While some will object to the idea of a Vietnam picture with such a rah-rah , patriotic tone , Soldiers ultimately achieves its main strategic objective : dramatizing the human cost of the conflict that came to define a generation .
Even if you do n't know the band or the album 's songs by heart , you will enjoy seeing how both evolve , and you will also learn a good deal about the state of the music business in the 21st Century .
The solid filmmaking and convincing characters makes this a high water mark for this genre .
Films about loss , grief and recovery are pretty valuable these days .
Seen in that light , Moonlight Mile should strike a nerve in many .
It 's endlessly inventive , consisten

With the same sort of good-natured fun found in films like Tremors , Eight Legged Freaks is prime escapist fare .
A sharp , amusing study of the cult of celebrity .
The sentimental cliches mar an otherwise excellent film .
A powerful performance from Mel Gibson and a brutal 90-minute battle sequence that does everything but issue you a dog-tag and an M-16 .
A graceful , moving tribute to the courage of New York 's finest and a nicely understated expression of the grief shared by the nation at their sacrifice .
A coming-of-age tale from New Zealand whose boozy , languid air is balanced by a rich visual clarity and deeply felt performances across the board .
Made to be Jaglomized is the Cannes Film Festival , the annual Riviera spree of flesh , buzz , blab and money .
The charming result is Festival in Cannes .
If you 're looking for something new and hoping for something entertaining , you 're in luck .
A hugely rewarding experience that 's every bit as enlightening , insightful and ent

...

If you can get past the fantastical aspects and harsh realities of `` The Isle '' you 'll get a sock-you-in-the-eye flick that is a visual tour-de-force and a story that is unlike any you will likely see anywhere else .
There are as many misses as hits , but ultimately , it finds humor in the foibles of human behavior , and it 's a welcome return to the roots of a genre that should depend on surprises .
A well-made thriller with a certain level of intelligence and non-reactionary morality .
There 's enough science to make it count as educational , and enough beauty to make it unforgettable .
Makmalbaf follows a resolutely realistic path in this uncompromising insight into the harsh existence of the Kurdish refugees of Iran 's borderlands .
For a good chunk of its running time , Trapped is an effective and claustrophobic thriller .
Most of Crush is a clever and captivating romantic comedy with a welcome pinch of tartness .
Nair does capture the complexity of a big family and its trials 

This is a film brimming with detail and nuance and one that speaks volumes about the ability of the human spirit to find solace in events that could easily crush it forever .
The director , Steven Shainberg , has succeeded by focusing intently on his characters , making them quirky individuals rather than figures of fun .
The movie has a soft , percolating magic , a deadpan suspense .
A well-made and often lovely depiction of the mysteries of friendship .
The Transporter is as lively and as fun as it is unapologetically dumb

There has been much puzzlement among critics about what the election symbolizes .
An enjoyable film for the family , amusing and cute for both adults and kids .
The second chapter of the Harry Potter series is even more magical than the first and simply the best family film of the year .
More honest about Alzheimer 's disease , I think , than Iris .
The acting alone is worth the price of admission .
An excellent romp that boasts both a heart and a mind .
Interacti

Although it 's a bit smug and repetitive , this documentary engages your brain in a way few current films do .
Demands too much of most viewers .
The story drifts so inexorably into cliches about tortured ( and torturing ) artists and consuming but impossible love that you ca n't help but become more disappointed as each overwrought new sequence plods on .
It should be mentioned that the set design and interiors of the haunted vessel are more than effectively creepy and moodily lit .
Shamelessly sappy and , worse , runs away from its own provocative theme .
The Ring just left me cold and wet like I was out in the Seattle drizzle without rainwear .
The film seems a dead weight .
The lack of pace kills it , although , in a movie about cancer , this might be apt .
For anyone who grew up on Disney 's 1950 Treasure Island , or remembers the 1934 Victor Fleming classic , this one feels like an impostor .
A clutchy , indulgent and pretentious travelogue and diatribe against ... well , just st

The Piano Teacher is the sort of movie that discourages American audiences from ever wanting to see another foreign film .
So what is the point ?
Lovingly choreographed bloodshed taking place in a pristine movie neverland , basically .
This is junk food cinema at its greasiest .
When it 's all wet , Blue Crush is highly enjoyable .
When it 's on dry land , though , this surfer-girl melodrama starts gasping like a beached grouper .
Most new movies have a bright sheen .
Some , like Ballistic , arrive stillborn ... looking like the beaten , well-worn video box cover of seven years into the future .
The story is naturally poignant , but first-time screenwriter Paul Pender overloads it with sugary bits of business .
Choose your reaction : A. ) That sure is funny !
B. ) That sure is pathetic !
A sermonizing and lifeless paean to teenage dullards .
Before long , the film starts playing like General Hospital crossed with a Saturday Night Live spoof of Dog Day Afternoon .
The charms of willful 

The filmmakers juggle and juxtapose three story lines but fail to come up with one cogent point , unless it 's that life stinks , especially for sensitive married women who really love other women .
The movie feels like it 's going to be great , and it carries on feeling that way for a long time , but takeoff just never happens .
A gimmick in search of a movie : how to get Carvey into as many silly costumes and deliver as many silly voices as possible , plot mechanics be damned .
... the last time I saw a theater full of people constantly checking their watches was during my SATs .
... fifty minutes of tedious adolescent melodramatics followed by thirty-five minutes of inflated nonsense .
It 's often faintly amusing , but the problems of the characters never become important to us , and the story never takes hold .
There 's no denying the elaborateness of the artist 's conceptions , nor his ability to depict them with outrageous elan , but really the whole series is so much pretentious

In [51]:
# Common Setup
ps = PredefinedSplit([-1 for s in train_sentences] + [0 for s in dev_sentences])
ps_coarse = PredefinedSplit([-1 for s in train_sentences_coarse] + [0 for s in dev_sentences_coarse])

In [52]:
# Vectorizing

count_Xs = count_vectorize(train_sentences + dev_sentences)
count_test_Xs = count_vectorize(test_sentences)

bigram_Xs = count_vectorize(train_sentences + dev_sentences, ngram=True)
bigram_test_Xs = count_vectorize(test_sentences, ngram=True)

# Fit and then vectorize on coarse sentences
count_vectorizer.fit(train_sentences_coarse)
tuple_count_vectorizer.fit(train_sentences_coarse)

count_Xs_coarse = count_vectorize(train_sentences_coarse + dev_sentences_coarse)
count_test_Xs_coarse = count_vectorize(test_sentences_coarse) 

bigram_Xs_coarse = count_vectorize(train_sentences_coarse + dev_sentences_coarse, ngram=True)
bigram_test_Xs_coarse = count_vectorize(test_sentences_coarse, ngram=True)

In [ ]:
def test_model(underlying_model, param_grid, cv=ps, fine=False, bigrams=False):
    if not fine:
        train_y, dev_y, test_y = coarse_train_y, coarse_dev_y, coarse_test_y
    else:
        train_y, dev_y, test_y = fine_train_y, fine_dev_y, fine_test_y
    if bigrams:
        Xs, test_Xs = bigram_Xs, bigram_test_Xs
    else:
        Xs, test_Xs = count_Xs, count_test_Xs
    grid = GridSearchCV(underlying_model, param_grid, cv=ps)
    grid.fit(Xs, train_y + dev_y)
    
    print(grid.best_params_)
    print(grid.score(test_Xs, test_y))

In [ ]:
# Naive Bayes
nb_grid = {"alpha": [1e-4, 0.01, 0.1, 1.0, 2.0, 10.0]}


bnb = BernoulliNB()
test_model(bnb, nb_grid)
test_model(bnb, nb_grid, fine=True)

test_model(bnb, nb_grid, bigrams=True)
test_model(bnb, nb_grid, bigrams=True, fine=True)

print("")

mnb = MultinomialNB()
test_model(mnb, nb_grid)
test_model(mnb, nb_grid, fine=True)

test_model(mnb, nb_grid, bigrams=True)
test_model(mnb, nb_grid, bigrams=True, fine=True)

In [ ]:
# SVM
svm_grid = {
        "C": [1e-4, 0.01, 0.1, 1.0, 2.0, 10.0], 
        "tol": [1e-4, 0.01, 0.1, 1.0, 2.0, 10.0],
        "max_iter": range(1000, 10001, 1000)
}

svm = LinearSVC()
test_model(svm, svm_grid)
test_model(svm, svm_grid, fine=True)

In [ ]:
# Random Forest
rf_grid = {
    "n_estimators": [10, 20, 30, 40, 50],
    "criterion": ["gini", "entropy"],
    "max_depth": [5, 10, 50, 100, 1000]
}

rf = RandomForestClassifier()
test_model(rf, rf_grid)
test_model(rf, rf_grid, fine=True)

In [ ]:
# Logistic Regression
lr_grid = {
    "penalty": ["l1", "l2"],
    "tol": [1e-4, 0.01, 0.1, 1.0, 2.0, 10.0],
    "C": [1e-4, 0.01, 0.1, 1.0, 2.0, 10.0]
}

lr = LogisticRegression()
test_model(lr, lr_grid)
test_model(lr, lr_grid, fine=True)

In [ ]:
# K-Nearest Neighbors
kn_grid = {
    "n_neighbors": [5, 10, 15, 20],
    "weights": ["uniform", "distance"]
}

kn = KNeighborsClassifier()
test_model(kn, kn_grid)
test_model(kn, kn_grid, fine=True)

In [53]:
# Word Vector Averaging

import warnings
warnings.filterwarnings("ignore")

def review_wordlist(review, remove_stopwords=False):
    review_text = review
    # 2. Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # 3. Converting to lower case and splitting
    words = review_text.lower().split()
    # 4. Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return words

num_features = 125  # Word vector dimensionality
min_word_count = 5 # Minimum word count
num_workers = 2     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

tokenized_sentences_fine = []
tokenized_sentences_coarse = []

for sentence in list(train_sentences + dev_sentences):
    tokenized_sentences_fine.append(review_wordlist(sentence, remove_stopwords=False))
    
for sentence in list(train_sentences_coarse + dev_sentences_coarse):
    tokenized_sentences_coarse.append(review_wordlist(sentence, remove_stopwords=False))

model_fine = word2vec.Word2Vec(tokenized_sentences_fine, min_count=min_word_count, size=num_features, sg=1, iter=5)
model_coarse = word2vec.Word2Vec(tokenized_sentences_coarse, min_count=min_word_count, size=num_features, sg=1, iter=5)

model_fine.train(tokenized_sentences_fine, total_examples=len(tokenized_sentences_fine), epochs=10)
model_coarse.train(tokenized_sentences_coarse, total_examples=len(tokenized_sentences_coarse), epochs=10)


# Function to average all word vectors in a review
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    if nwords == 0:
        return featureVec
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# Function for calculating average word vectors for all reviews
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:

        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs
    
trainDataVecs_fine = getAvgFeatureVecs(tokenized_sentences_fine, model_fine, num_features)
trainDataVecs_coarse = getAvgFeatureVecs(tokenized_sentences_coarse, model_coarse, num_features)

filtered_test_reviews_fine = []
filtered_test_reviews_coarse = []

for review in test_sentences:
    filtered_test_reviews_fine.append(review_wordlist(review, remove_stopwords=False))
    
for review in test_sentences_coarse:
    filtered_test_reviews_coarse.append(review_wordlist(review, remove_stopwords=False))
    
testDataVecs_fine = getAvgFeatureVecs(filtered_test_reviews_fine, model_fine, num_features)
testDataVecs_coarse = getAvgFeatureVecs(filtered_test_reviews_coarse, model_coarse, num_features)

clf = LinearSVC()
clf.fit(trainDataVecs_coarse, coarse_train_y + coarse_dev_y)
print(clf.score(testDataVecs_coarse, coarse_test_y))

# grid = GridSearchCV(LinearSVC(), param_grid = svm_grid, cv=ps)
# grid.fit(trainDataVecs, coarse_train_y + coarse_dev_y)

# print("Best params with VecAvg representation and linear SVM (coarse labels):", grid.best_params_)
# print('Optimal accuracy score with VecAvg representation and linear SVM (coarse labels):', grid.score(testDataVecs, coarse_test_y))

# grid.fit(trainDataVecs, fine_train_y + fine_dev_y)
# print("Best params with VecAvg representation and linear SVM (coarse labels):", grid.best_params_)
# print('Optimal accuracy score with VecAvg representation and linear SVM (fine labels):', grid.score(testDataVecs, fine_test_y))

# grid = GridSearchCV(LogisticRegression(), param_grid = svm_grid, cv=ps)
# grid.fit(trainDataVecs, coarse_train_y + coarse_dev_y)

# print("Best params with VecAvg representation and SoftMax (coarse labels):", grid.best_params_)
# print('Optimal accuracy score with VecAvg representation and SoftMax (coarse labels):', grid.score(testDataVecs, coarse_test_y))

# grid.fit(trainDataVecs, fine_train_y + fine_dev_y)
# print("Best params with VecAvg representation and SoftMax (coarse labels):", grid.best_params_)
# print('Optimal accuracy score with VecAvg representation and SoftMax (fine labels):', grid.score(testDataVecs, fine_test_y))

0.7116968698517299
